In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import cv2
from kerastuner.tuners import RandomSearch
import kerastuner as kt
from kerastuner import HyperModel


In [30]:
x_train=np.load('250_img.npy')
y_train=np.load('250_gender.npy').reshape(-1,1)

In [33]:
y_train=y_train.astype(int)

In [14]:
class my_hyper(HyperModel):
    def __init__(self,num_classes):
        self.num_classes=num_classes
    
    def build(self,hp):
        num_filters=hp.Int('filters',min_value=32,max_value=128,step=32)
        num_pools=hp.Int('pool_size',min_value=2,max_value=8,step=2)
        
        inputs=keras.layers.Input(shape=(250,250,1),name='inputs')
        common=keras.layers.Conv2D(filters=num_filters,kernel_size=7,activation='relu')(inputs)
        common=keras.layers.MaxPooling2D(pool_size=num_pools)(common)
        common=keras.layers.Conv2D(filters=num_filters,kernel_size=5,activation='relu')(common)
        common=keras.layers.MaxPooling2D(pool_size=num_pools)(common)
        common=keras.layers.Dropout(0.2)(common)
        
        flat=keras.layers.Flatten()(common)
        
        num_dense=hp.Int('units',min_value=32,max_value=128,step=32)
        
        gender_layer=keras.layers.Dropout(0.2)(flat)
        gender_layer=keras.layers.Dense(units=num_dense,activation='relu',kernel_initializer='he_normal')(gender_layer)
        gender_layer=keras.layers.Dropout(0.3)(gender_layer)
        gender_layer=keras.layers.Dense(units=num_dense,activation='relu',kernel_initializer='he_normal')(gender_layer)
        gender_layer=keras.layers.Dense(8,activation='relu',kernel_initializer='he_normal')(gender_layer)
        gender_layer=keras.layers.Dense(self.num_classes,activation='sigmoid')(gender_layer)
        
        num_learning_rate=hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])
        
        model=keras.Model(inputs=inputs,outputs=[gender_layer])
        model.compile(loss='binary_crossentropy',optimizer=keras.optimizers.Adam(learning_rate=num_learning_rate),
                     metrics=['accuracy'])
        
        return model    

In [15]:
tune_model=my_hyper(1)

In [18]:
tuner=kt.Hyperband(tune_model,
                  objective='val_accuracy',
                   max_epochs=10,
                   directory='my_dir',
                   project_name='gender',
                  )


In [20]:
from tensorflow.keras.callbacks import TensorBoard
import time
name='gender_CNN---{}'.format(int(time.time()))
tensorboard=TensorBoard(log_dir='gender_logs\{}'.format(name))

In [22]:
import IPython
class clear_output(tf.keras.callbacks.Callback):
    def on_train_end(*args,**kwargs):
        IPython.display.clear_output(wait=True)
    

In [35]:
tuner.search(x_train,y_train,epochs=10,validation_split=0.2,callbacks=[clear_output()])

Epoch 1/2
  1/593 [..............................] - ETA: 0s - loss: 9.6138 - accuracy: 0.5000

KeyboardInterrupt: 